# Cap7: Inteligencia Artificial Generativa


## Primer punto:


### Ejercicio #1: Generación de texto con GPT-3

Para este primer punto, hemos implementado un traductor con una API gratuita. El acceso a la API se hace a través de la variable GROQ_API_KEY.

Es de notar que el uso no está limitado a traducir texto.

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from groq import Groq

client = Groq(
    api_key=userdata.get('GROQ_API_KEY')
)

def translate(idioma_entrada, idioma_salida, prompt):

  completion = client.chat.completions.create(
      model="llama-3.3-70b-versatile",
      messages=[
          {
              "role": "user",
              "content": "Translate from %s to %s: %s" % (idioma_entrada, idioma_salida, prompt)
          }
      ],
      temperature=1,
      max_completion_tokens=1024,
      top_p=1,
      stream=True,
      stop=None,
  )

  for chunk in completion:
      print(chunk.choices[0].delta.content or "", end="")


idioma_entrada = input("Seleccione el idioma de entrada: ")
idioma_salida = input("Seleccione el idioma de salida: ")
prompt = input("%s -> %s\n" % (idioma_entrada, idioma_salida))

translate(idioma_entrada, idioma_salida, prompt)




Seleccione el idioma de entrada: Español
Seleccione el idioma de salida: Japones
Español -> Japones
Hola Mundo!!
Konnichiwa Sekai!!

### Ejercicio #4: Resumen automático de texto

Este código implementa un modelo Transformer para la tarea de resumen automático de texto.
Se carga un dataset de textos largos con sus resúmenes, y el modelo se entrena desde cero para aprender
a generar resúmenes concisos y precisos.

Modificaciones clave del archivo original:

1. Se ajustó el decoder para generar texto más corto en lugar de traducción palabra por palabra.
2. Se usa una función de pérdida adaptada a la tarea de resumen, evitando penalizar frases más cortas.
3. Se establecen restricciones en la generación de texto para limitar la longitud del resumen.
4. Se carga el dataset público CNN/DailyMail y se preprocesan los textos.
5. Se tokenizan los datos para entrenar el modelo Transformer.
6. Se entrena el modelo utilizando PyTorch con una función de pérdida basada en CrossEntropyLoss y optimizador Adam.
7. Se evalúa el modelo con métricas de ROUGE para medir la calidad del resumen generado.
8. Se agregan gráficos para visualizar la evolución de las métricas ROUGE durante el entrenamiento.

El modelo se entrena en un dataset de noticias con resúmenes y se evalúa su desempeño en ejemplos reales.

In [ ]:
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install -U bitsandbytes



import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from datasets import load_dataset
from evaluate import load
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from huggingface_hub import login
login()  # Esto te pedirá ingresar tu token

In [ ]:
# Desactivar Weights & Biases (W&B)
os.environ["WANDB_DISABLED"] = "true"


# Cargar el dataset público CNN/DailyMail
dataset = load_dataset("cnn_dailymail", "3.0.0", split={"train": "train[:10%]", "test": "test[:10%]"})


# Cargar el tokenizer para procesar los textos
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Función para tokenizar los datos
def preprocess_function(examples):
    model_inputs = tokenizer(examples["article"], max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = torch.tensor(labels["input_ids"], dtype=torch.long)
    return model_inputs

# Preprocesar los datos
dataset = dataset.map(preprocess_function, batched=True)

# Dividir en entrenamiento y prueba
train_data = dataset["train"]

#train_data = train_data.select(range(300))

test_data = dataset["test"]

#test_data= test_data.select(range(300))



Map:   0%|          | 0/28711 [00:00<?, ? examples/s]

Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

In [ ]:

# Cargar el modelo Transformer para resumen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base").to(device)
model.gradient_checkpointing_enable()



# Configurar los argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    save_steps=2000,
    save_total_limit=2,
    logging_dir="./logs",
    fp16=True,
    optim="adamw_torch",
)

# Evaluar el modelo con ROUGE
metric = load("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {key: value.mid.fmeasure for key, value in result.items()}


# Definir el Trainer para entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Iniciar el entrenamiento
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluación en el conjunto de prueba
eval_results = trainer.evaluate()
print("Resultados de evaluación:", eval_results)

In [ ]:
# Visualizar métricas ROUGE
rouge_keys = ["rouge1", "rouge2", "rougeL"]
scores = [eval_results[key] for key in rouge_keys]

plt.figure(figsize=(8, 5))
plt.bar(rouge_keys, scores, color=["blue", "green", "red"])
plt.xlabel("Métricas ROUGE")
plt.ylabel("Puntaje F1")
plt.title("Resultados de Evaluación con ROUGE")
plt.ylim(0, 1)
plt.show()

print("Entrenamiento y evaluación completados.")